In [ ]:
import pandas as pd

In [ ]:
# Import NY later
# ny_listings = pd.read('Data/New_York_City.csv')
denver_listings = pd.read_csv('Data/Denver.csv')
# austin_listings = pd.read('Data/Austin.csv')
sf_listings = pd.read_csv('Data/San_Francisco.csv')

In [ ]:
# Add city column
denver_listings['city'] = 'Denver'
sf_listings['city'] = 'San Francisco'

In [ ]:
# Merge all city datasets
listings_raw = pd.merge(denver_listings,sf_listings,how='outer')
listings_raw.head()

In [ ]:
# See all columns in merged dataset
listings_raw.columns

In [ ]:
# Keep relevant variables
listings = pd.DataFrame(listings_raw[['id','host_acceptance_rate',
       'host_is_superhost', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'amenities', 'price',
       'minimum_nights', 'maximum_nights',
       'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d',  
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value',
       'reviews_per_month',
       'city']])
listings.head()

In [ ]:
# Drop all rows with null values
listings.dropna(inplace=True)
listings

In [ ]:
# Convert t/f to 1/0
listings['host_is_superhost'] = listings['host_is_superhost']\
                                .apply(lambda value: 1 if value == 't' else 0)
listings.head()

In [ ]:
listings['price']

In [ ]:
# Convert price variable to float
listings['price_int'] = listings['price'].str.replace('$', '')
listings['price_int'] = listings['price_int'].str.replace(',', '')
listings['price_int'] = listings['price_int'].astype(float)
listings['price_int'].dtype

In [ ]:
# listings.to_csv('Data/Consolidated_Data.csv')

In [ ]:
# Create model dataset
model_data = pd.get_dummies(listings[['price_int','accommodates','bedrooms','review_scores_rating','room_type','city']])
model_data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Assign X and y
y = model_data['price_int']
X = model_data.drop(columns='price_int').values

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train,y_train)
print(f'Model train score: {model.score(X_train,y_train)}')
print(list(y_test[0:10]))
print(list(model.predict(X_test[0:10])))

In [ ]:
model_data.corr()

In [ ]:
model_data_1 = model_data[['price_int','bedrooms','accommodates','room_type_Entire home/apt']]

# Assign X and y
y = model_data_1['price_int']
X = model_data_1.drop(columns='price_int').values

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

model_1 = LinearRegression()
model_1.fit(X_train,y_train)
model_1.score(X_train,y_train)